In [ ]:
from tensorflow.keras.layers import Attention, Dropout, GroupNormalization, BatchNormalization, Flatten, Dense, Conv1D, MaxPooling1D,InputLayer, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import sklearn.linear_model
from PIL import Image
import tensorflow as tf
from google.colab import drive
from PIL import Image
drive.mount('/content/drive/')
DIR = 'drive/MyDrive/SHTEM24PAPAYA/'

Mounted at /content/drive/


In [ ]:
def padding(small, img_back, hi = 1/5, vi = 1/5):

  small_dims = small.shape
  big_dims = img_back.shape
  horiz_pad = big_dims[1]-small_dims[1]
  vert_pad = big_dims[0]-small_dims[0]
  left_pad = int(np.round(horiz_pad*hi,0))
  up_pad = int(np.round(vert_pad*vi,0))
  padded = np.zeros_like(img_back)
  padded[up_pad:up_pad+small_dims[0], left_pad:left_pad+small_dims[1]] = small
  return padded

In [ ]:
x = np.load(DIR+'350x.npy')
y = np.load(DIR+'350y.npy')

In [ ]:
#WITHOUT BACTHNORM
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.applications import MobileNetV2

def build_mobilenetv2_model(k, f, input_shape=(512, 512, 3)):

    base_model =  MobileNetV2 (weights='imagenet', include_top=False, input_shape=input_shape)

    base_model.trainable = False
    encoder_output = base_model.output

    up1 = layers.UpSampling2D((2, 2))(encoder_output)
    conv1 = layers.Conv2D(f*2, (k, k), activation='leaky_relu', padding='same')(up1)
    conv1 = layers.Conv2D(f*4, (k, k), activation='leaky_relu', padding='same')(conv1)
    up2 = layers.UpSampling2D((2, 2))(conv1)
    conv2 = layers.Conv2D(f*2, (k, k), activation='leaky_relu', padding='same')(up2)
    conv2 = layers.Conv2D(f*4, (k, k), activation='leaky_relu', padding='same')(conv2)
    up3 = layers.UpSampling2D((2, 2))(conv2)
    conv3 = layers.Conv2D(f*2, (k, k), activation='leaky_relu', padding='same')(up3)
    up4 = layers.UpSampling2D((2, 2))(conv3)
    conv4 = layers.Conv2D(f*8, (k, k), activation='leaky_relu', padding='same')(up4)
    up5 = layers.UpSampling2D((2, 2))(conv4)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(up5)
    model = Model(inputs=base_model.input, outputs=outputs)
    return model

for k in [3,4,5]:
    for f in [2,4,8]:
        mnetv2Model = build_mobilenetv2_model(k,f)

        mnetv2Model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=[tf.keras.metrics.BinaryIoU(
            target_class_ids=(0, 1), threshold=0.5, name=None, dtype=None
            )])

        mnetv2Model.summary()

        trainnum = int(len(x)*0.9)
        e= mnetv2Model.fit(x[:trainnum], y[:trainnum], validation_data=(x[trainnum:], y[trainnum:]), epochs=5, batch_size=32)

        pd.DataFrame(e.history).to_csv(DIR+'Histories/mnetv2_leakyrelu_k'+str(k)+'_f'+str(f)+'_350_nobatchnorm.csv')

9406464/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 256, 256, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 256, 256, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (N

KeyboardInterrupt: 

In [ ]:
#WITH BACTHNORM
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.applications import MobileNetV2

def build_mobilenetv2_model(k, f, input_shape=(512, 512, 3)):

    base_model =  MobileNetV2 (weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    encoder_output = base_model.output

    up1 = layers.UpSampling2D((2, 2))(encoder_output)
    conv1 = layers.Conv2D(f, (k, k), activation='leaky_relu', padding='same')(up1)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Conv2D(f*2, (k, k), activation='leaky_relu', padding='same')(conv1)
    conv1 = layers.BatchNormalization()(conv1)
    up2 = layers.UpSampling2D((2, 2))(conv1)
    conv2 = layers.Conv2D(f*2, (k, k), activation='leaky_relu', padding='same')(up2)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Conv2D(f*4, (k, k), activation='leaky_relu', padding='same')(conv2)
    conv2 = layers.BatchNormalization()(conv2)
    up3 = layers.UpSampling2D((2, 2))(conv2)
    conv3 = layers.Conv2D(f*4, (k, k), activation='leaky_relu', padding='same')(up3)
    conv3 = layers.BatchNormalization()(conv3)
    up4 = layers.UpSampling2D((2, 2))(conv3)
    conv4 = layers.Conv2D(f*4, (k, k), activation='leaky_relu', padding='same')(up4)
    conv4 = layers.BatchNormalization()(conv4)
    up5 = layers.UpSampling2D((2, 2))(conv4)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(up5)
    model = Model(inputs=base_model.input, outputs=outputs)
    return model

for k in [3,4,5]:
    for f in [2,4,8]:
        mnetv2Model = build_mobilenetv2_model(k,f)

        mnetv2Model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=[tf.keras.metrics.BinaryIoU(
            target_class_ids=(0, 1), threshold=0.5, name=None, dtype=None
            )])

        mnetv2Model.summary()

        trainnum = int(len(x)*0.9)
        e= mnetv2Model.fit(x[:trainnum], y[:trainnum], validation_data=(x[trainnum:], y[trainnum:]), epochs=5, batch_size=32)

        pd.DataFrame(e.history).to_csv(DIR+'Histories/mnetv2_leakyrelu_k'+str(k)+'_f'+str(f)+'_350_batchnorm.csv')

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 256, 256, 32)         864       ['input_10[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 256, 256, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 256, 256, 32)         0         ['bn_Conv1[0][0]']      

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 256, 256, 32)         864       ['input_11[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 256, 256, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 256, 256, 32)         0         ['bn_Conv1[0][0]']     

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_12 (InputLayer)       [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 256, 256, 32)         864       ['input_12[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 256, 256, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 256, 256, 32)         0         ['bn_Conv1[0][0]']     

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 256, 256, 32)         864       ['input_13[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 256, 256, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 256, 256, 32)         0         ['bn_Conv1[0][0]']     

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_14 (InputLayer)       [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 256, 256, 32)         864       ['input_14[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 256, 256, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 256, 256, 32)         0         ['bn_Conv1[0][0]']     

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 256, 256, 32)         864       ['input_15[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 256, 256, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 256, 256, 32)         0         ['bn_Conv1[0][0]']     

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_16 (InputLayer)       [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 256, 256, 32)         864       ['input_16[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 256, 256, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 256, 256, 32)         0         ['bn_Conv1[0][0]']     

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_17 (InputLayer)       [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 256, 256, 32)         864       ['input_17[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 256, 256, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 256, 256, 32)         0         ['bn_Conv1[0][0]']     

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_18 (InputLayer)       [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 256, 256, 32)         864       ['input_18[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 256, 256, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 256, 256, 32)         0         ['bn_Conv1[0][0]']     